In [1]:
pip install requests

In [2]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych zwierząt (wartość w tys. zł)
animal_variables = {
    '151881': 'Jelenie',
    '151880': 'Sarny',
    '151879': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Ściągnięcie danych XML
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Przetworzenie danych XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (tys zł)": val
            })

    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Połączenie wszystkich ramek danych skupu zwierząt (w zł)
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Wyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_value.csv", index=False)

Fetching data for Jelenie (variable ID: 151881)
Fetching data for Sarny (variable ID: 151880)
Fetching data for Dziki (variable ID: 151879)
         Region  Year  Jelenie (tys zł)  Sarny (tys zł)  Dziki (tys zł)
0  DOLNOŚLĄSKIE  2014            3564.1          3711.1          1439.4
1  DOLNOŚLĄSKIE  2015            3847.4          3625.1          2648.0
2  DOLNOŚLĄSKIE  2016            4915.7          4655.0          2974.6
3  DOLNOŚLĄSKIE  2017            4875.6          3759.7          2678.4
4  DOLNOŚLĄSKIE  2018            5493.7          3456.5          1289.2
             Region  Year  Jelenie (tys zł)  Sarny (tys zł)  Dziki (tys zł)
171  ŚWIĘTOKRZYSKIE  2020             215.1           482.4            25.4
172  ŚWIĘTOKRZYSKIE  2021             217.2           560.8            57.1
173  ŚWIĘTOKRZYSKIE  2022            1218.2          1540.2           313.3
174  ŚWIĘTOKRZYSKIE  2023            1015.7          1746.4           456.8
175  ŚWIĘTOKRZYSKIE  2024             750.2     

In [3]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych zwierząt (ilość w tonach)
animal_variables = {
    '151877': 'Jelenie',
    '151884': 'Sarny',
    '151883': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Ściągnięcie danych XML
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Przetworzenie danych XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (tony)": val
            })

    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Połączenie wszystkich ramek danych skupu zwierząt (w tonach)
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Wyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_tons.csv", index=False)

Fetching data for Jelenie (variable ID: 151877)
Fetching data for Sarny (variable ID: 151884)
Fetching data for Dziki (variable ID: 151883)
         Region  Year  Jelenie (tony)  Sarny (tony)  Dziki (tony)
0  DOLNOŚLĄSKIE  2014           473.0         266.0         354.0
1  DOLNOŚLĄSKIE  2015           466.0         247.0         548.0
2  DOLNOŚLĄSKIE  2016           535.0         285.0         541.0
3  DOLNOŚLĄSKIE  2017           504.0         271.0         620.0
4  DOLNOŚLĄSKIE  2018           575.0         269.0         443.0
             Region  Year  Jelenie (tony)  Sarny (tony)  Dziki (tony)
171  ŚWIĘTOKRZYSKIE  2020            39.0          45.0           8.0
172  ŚWIĘTOKRZYSKIE  2021            35.0          38.0          10.0
173  ŚWIĘTOKRZYSKIE  2022           134.0          78.0          43.0
174  ŚWIĘTOKRZYSKIE  2023           109.0          84.0          52.0
175  ŚWIĘTOKRZYSKIE  2024           110.0          81.0          32.0


In [4]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych zwierząt (odstrzał w sztukach)
animal_variables = {
    '4540': 'Jelenie',
    '4541': 'Sarny',
    '4542': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Ściągnięcie danych XML
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Przetworzenie danych XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (odstrzał sztuki)": val
            })

    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Połączenie wszystkich ramek danych odstrzału zwierząt
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Wyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_hunted.csv", index=False)

Fetching data for Jelenie (variable ID: 4540)
Fetching data for Sarny (variable ID: 4541)
Fetching data for Dziki (variable ID: 4542)
         Region  Year  Jelenie (odstrzał sztuki)  Sarny (odstrzał sztuki)  \
0  DOLNOŚLĄSKIE  2014                     7013.0                  17973.0   
1  DOLNOŚLĄSKIE  2015                     7607.0                  18448.0   
2  DOLNOŚLĄSKIE  2016                     7788.0                  19808.0   
3  DOLNOŚLĄSKIE  2017                     8969.0                  20667.0   
4  DOLNOŚLĄSKIE  2018                     9087.0                  20239.0   

   Dziki (odstrzał sztuki)  
0                  30210.0  
1                  38044.0  
2                  32381.0  
3                  39766.0  
4                  33844.0  
             Region  Year  Jelenie (odstrzał sztuki)  Sarny (odstrzał sztuki)  \
171  ŚWIĘTOKRZYSKIE  2020                     1194.0                   5595.0   
172  ŚWIĘTOKRZYSKIE  2021                      388.0               

In [5]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych zwierząt (ogólna ilość w tys. sztuk)
animal_variables = {
    '4530': 'Jelenie',
    '4531': 'Sarny',
    '4532': 'Dziki'
}

all_animals_dfs = []

for var_id, animal_name in animal_variables.items():
    print(f"Fetching data for {animal_name} (variable ID: {var_id})")
    all_records_for_animal = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Ściągnięcie danych XML
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Przetworzenie danych XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_animal.append({
                "Region": name,
                "Year": record_year,
                f"{animal_name} (ogółem tys. sztuk)": val
            })

    df_animal = pd.DataFrame(all_records_for_animal)
    all_animals_dfs.append(df_animal)

# Połączenie wszystkich ramek danych zwierząt ogółem
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_animals_dfs[0]
for i in range(1, len(all_animals_dfs)):
    df_merged = pd.merge(df_merged, all_animals_dfs[i], on=['Region', 'Year'], how='outer')

# Wyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_all_animals_overall_amount.csv", index=False)

Fetching data for Jelenie (variable ID: 4530)
Fetching data for Sarny (variable ID: 4531)
Fetching data for Dziki (variable ID: 4532)
         Region  Year  Jelenie (ogółem tys. sztuk)  Sarny (ogółem tys. sztuk)  \
0  DOLNOŚLĄSKIE  2014                         17.1                       83.9   
1  DOLNOŚLĄSKIE  2015                         15.1                       82.5   
2  DOLNOŚLĄSKIE  2016                         15.6                       85.1   
3  DOLNOŚLĄSKIE  2017                         32.7                       80.7   
4  DOLNOŚLĄSKIE  2018                         29.4                       79.0   

   Dziki (ogółem tys. sztuk)  
0                       27.9  
1                       23.1  
2                       22.4  
3                       29.9  
4                       19.4  
             Region  Year  Jelenie (ogółem tys. sztuk)  \
171  ŚWIĘTOKRZYSKIE  2020                          4.5   
172  ŚWIĘTOKRZYSKIE  2021                          4.6   
173  ŚWIĘTOKRZYSKIE

In [6]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych obszarów pożarów leśnych (POWIERZCHNIA W HA)
fire_variables = {
    '151910': 'Powierzchnia pożarów lasów (ha)'
}

all_fire_dfs = []

for var_id, variable_name in fire_variables.items():
    print(f"Fetching data for {variable_name} (variable ID: {var_id})")
    all_records_for_variable = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Ściągnięcie danych XML
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Przetworzenie danych XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_variable.append({
                "Region": name,
                "Year": record_year,
                f"{variable_name}": val
            })

    df_current_variable = pd.DataFrame(all_records_for_variable)
    all_fire_dfs.append(df_current_variable)

# Połączenie wszystkich ramek danych pożarów leśnych
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_fire_dfs[0]
for i in range(1, len(all_fire_dfs)):
    df_merged = pd.merge(df_merged, all_fire_dfs[i], on=['Region', 'Year'], how='outer')

# DWyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_forest_fires_lands.csv", index=False)

Fetching data for Powierzchnia pożarów lasów (ha) (variable ID: 151910)
               Region  Year  Powierzchnia pożarów lasów (ha)
0         MAŁOPOLSKIE  2014                            71.46
1             ŚLĄSKIE  2014                           205.22
2            LUBUSKIE  2014                            30.91
3       WIELKOPOLSKIE  2014                           118.41
4  ZACHODNIOPOMORSKIE  2014                            43.34
             Region  Year  Powierzchnia pożarów lasów (ha)
171  ŚWIĘTOKRZYSKIE  2024                           109.43
172       LUBELSKIE  2024                            84.96
173    PODKARPACKIE  2024                           111.69
174       PODLASKIE  2024                            84.01
175     MAZOWIECKIE  2024                           324.33


In [7]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych gruntów leśnych (POWIERZCHNIA W HA)
forest_variables = {
    '1610493': 'Grunty zalesione',
    '1610500': 'Grunty niezalesione'
}

all_forest_dfs = []

for var_id, forest_land_name in forest_variables.items():
    print(f"Fetching data for {forest_land_name} (variable ID: {var_id})")
    all_records_for_forest = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Ściągnięcie danych XML
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Przetworzenie danych XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_forest.append({
                "Region": name,
                "Year": record_year,
                f"Forest Lands - {forest_land_name} (ha)": val
            })

    df_forest = pd.DataFrame(all_records_for_forest)
    all_forest_dfs.append(df_forest)

# Połączenie wszystkich ramek danych powierzchni gruntów leśnych
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_forest_dfs[0]
for i in range(1, len(all_forest_dfs)):
    df_merged = pd.merge(df_merged, all_forest_dfs[i], on=['Region', 'Year'], how='outer')

# Wyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_forest_land_total.csv", index=False)

Fetching data for Grunty zalesione (variable ID: 1610493)
Fetching data for Grunty niezalesione (variable ID: 1610500)
         Region  Year  Forest Lands - Grunty zalesione (ha)  \
0  DOLNOŚLĄSKIE  2017                             580912.31   
1  DOLNOŚLĄSKIE  2018                             581220.72   
2  DOLNOŚLĄSKIE  2019                             580812.35   
3  DOLNOŚLĄSKIE  2020                             581456.97   
4  DOLNOŚLĄSKIE  2021                             583111.79   

   Forest Lands - Grunty niezalesione (ha)  
0                                 13897.32  
1                                 13832.97  
2                                 14218.29  
3                                 14113.09  
4                                 13707.43  
             Region  Year  Forest Lands - Grunty zalesione (ha)  \
123  ŚWIĘTOKRZYSKIE  2020                             325284.75   
124  ŚWIĘTOKRZYSKIE  2021                             326182.68   
125  ŚWIĘTOKRZYSKIE  2022      

In [8]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL dla API
base_url_template = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{variable_id}?format=xml&unit-level=2&page=0&page-size=100"

# Zdefiniowanie zakresu lat
start_year = 2014
end_year = 2024

# Zdefiniowanie zmiennych dla różnych obszarów leśnych (POWIERZCHNIA W HA)
forest_variables = {
    '217916': 'Powierzchnia lasów ogółem'
}

all_forest_dfs = []

for var_id, forest_name in forest_variables.items():
    print(f"Fetching data for {forest_name} (variable ID: {var_id})")
    all_records_for_forest = []
    for year in range(start_year, end_year + 1):
        # Zbudowanie URL dla bieżącego roku i zmiennej
        url = base_url_template.format(variable_id=var_id) + f"&year={year}"

        # Download XML data
        response = requests.get(url)
        response.raise_for_status()  # upewnienie się, że zapytanie było udane
        xml_data = response.text

        # Parse the XML
        root = ET.fromstring(xml_data)

        # Wyodrębnienie odpowiednich danych
        for unit in root.findall(".//unitData"):
            name = unit.find("name").text
            val_element = unit.find(".//val")
            year_element = unit.find(".//year")

            val = float(val_element.text) if val_element is not None else None
            record_year = int(year_element.text) if year_element is not None else None

            all_records_for_forest.append({
                "Region": name,
                "Year": record_year,
                f"{forest_name} (ha)": val
            })

df_forest = pd.DataFrame(all_records_for_forest)
all_forest_dfs.append(df_forest)

# Połączenie wszystkich ramek danych powierzchni lasów
# Rozpoczęcie od pierwszej ramki danych i scalenie z nią kolejnych
df_merged = all_forest_dfs[0]
for i in range(1, len(all_forest_dfs)):
    df_merged = pd.merge(df_merged, all_forest_dfs[i], on=['Region', 'Year'], how='outer')

# Wyświetlenie i zapisanie danych
print(df_merged.head())
print(df_merged.tail())
df_merged.to_csv("bdl_data_2014-2024_forest_area_total.csv", index=False)

Fetching data for Powierzchnia lasów ogółem (variable ID: 217916)
               Region  Year  Powierzchnia lasów ogółem (ha)
0         MAŁOPOLSKIE  2014                       435291.75
1             ŚLĄSKIE  2014                       393849.16
2            LUBUSKIE  2014                       688140.44
3       WIELKOPOLSKIE  2014                       767530.76
4  ZACHODNIOPOMORSKIE  2014                       811876.28
             Region  Year  Powierzchnia lasów ogółem (ha)
171  ŚWIĘTOKRZYSKIE  2024                       331168.68
172       LUBELSKIE  2024                       589452.06
173    PODKARPACKIE  2024                       685594.97
174       PODLASKIE  2024                       631647.13
175     MAZOWIECKIE  2024                       831888.41


In [9]:
import pandas as pd

# Lista nazw plików CSV do połączenia
csv_files = [
    "bdl_data_2014-2024_all_animals_value.csv",
    "bdl_data_2014-2024_all_animals_tons.csv",
    "bdl_data_2014-2024_all_animals_hunted.csv",
    "bdl_data_2014-2024_all_animals_overall_amount.csv",
    "bdl_data_2014-2024_forest_fires_lands.csv",
    "bdl_data_2014-2024_forest_land_total.csv",
    "bdl_data_2014-2024_forest_area_total.csv"
]

# Wczytanie pierwszego pliku do zainicjowania połączonej ramki danych
df_combined = pd.read_csv(csv_files[0])

# Połączenie pozostałych plików
for i in range(1, len(csv_files)):
    df_current = pd.read_csv(csv_files[i])
    df_combined = pd.merge(df_combined, df_current, on=['Region', 'Year'], how='outer')

# Wyświetlenie pierwszych kilku wierszy połączonej ramki danych
print("Połączona ramka danych - pierwsze 5 wierszy:")
display(df_combined.head())

# Wyświetlenie ostatnich kilku wierszy połączonej ramki danych
print("\nPołączona ramka danych - ostatnie 5 wierszy:")
display(df_combined.tail())

# Zapisanie połączonej ramki danych do nowego pliku CSV
df_combined.to_csv("bdl_data_2014-2024_combined.csv", index=False)
print("\nPołączona ramka danych została zapisana do pliku: bdl_data_2014-2024_combined.csv")

Połączona ramka danych - pierwsze 5 wierszy:


,Region,Year,Jelenie (tys zł),Sarny (tys zł),Dziki (tys zł),Jelenie (tony),Sarny (tony),Dziki (tony),Jelenie (odstrzał sztuki),Sarny (odstrzał sztuki),Dziki (odstrzał sztuki),Jelenie (ogółem tys. sztuk),Sarny (ogółem tys. sztuk),Dziki (ogółem tys. sztuk),Powierzchnia pożarów lasów (ha),Forest Lands - Grunty zalesione (ha),Forest Lands - Grunty niezalesione (ha),Powierzchnia lasów ogółem (ha)
0,DOLNOŚLĄSKIE,2014,3564.1,3711.1,1439.4,473.0,266.0,354.0,7013.0,17973.0,30210.0,17.1,83.9,27.9,414.85,NaN,NaN,592815.56
1,DOLNOŚLĄSKIE,2015,3847.4,3625.1,2648.0,466.0,247.0,548.0,7607.0,18448.0,38044.0,15.1,82.5,23.1,238.62,NaN,NaN,593385.48
2,DOLNOŚLĄSKIE,2016,4915.7,4655.0,2974.6,535.0,285.0,541.0,7788.0,19808.0,32381.0,15.6,85.1,22.4,125.54,NaN,NaN,594968.12
3,DOLNOŚLĄSKIE,2017,4875.6,3759.7,2678.4,504.0,271.0,620.0,8969.0,20667.0,39766.0,32.7,80.7,29.9,40.86,580912.31,13897.32,594809.63
4,DOLNOŚLĄSKIE,2018,5493.7,3456.5,1289.2,575.0,269.0,443.0,9087.0,20239.0,33844.0,29.4,79.0,19.4,485.15,581220.72,13832.97,595053.69



Połączona ramka danych - ostatnie 5 wierszy:


,Region,Year,Jelenie (tys zł),Sarny (tys zł),Dziki (tys zł),Jelenie (tony),Sarny (tony),Dziki (tony),Jelenie (odstrzał sztuki),Sarny (odstrzał sztuki),Dziki (odstrzał sztuki),Jelenie (ogółem tys. sztuk),Sarny (ogółem tys. sztuk),Dziki (ogółem tys. sztuk),Powierzchnia pożarów lasów (ha),Forest Lands - Grunty zalesione (ha),Forest Lands - Grunty niezalesione (ha),Powierzchnia lasów ogółem (ha)
171,ŚWIĘTOKRZYSKIE,2020,215.1,482.4,25.4,39.0,45.0,8.0,1194.0,5595.0,4283.0,4.5,29.6,2.0,266.41,325284.75,5997.91,331282.66
172,ŚWIĘTOKRZYSKIE,2021,217.2,560.8,57.1,35.0,38.0,10.0,388.0,1944.0,1432.0,4.6,29.9,1.8,46.88,326182.68,5750.37,331933.05
173,ŚWIĘTOKRZYSKIE,2022,1218.2,1540.2,313.3,134.0,78.0,43.0,1426.0,5756.0,4894.0,4.9,30.1,1.8,241.03,326408.39,5547.87,331956.26
174,ŚWIĘTOKRZYSKIE,2023,1015.7,1746.4,456.8,109.0,84.0,52.0,1667.0,6090.0,5810.0,5.0,28.7,1.8,34.12,326080.15,5618.02,331698.17
175,ŚWIĘTOKRZYSKIE,2024,750.2,1190.7,202.7,110.0,81.0,32.0,1945.0,6185.0,4001.0,5.9,29.8,2.0,109.43,325518.43,5650.25,331168.68



Połączona ramka danych została zapisana do pliku: bdl_data_2014-2024_combined.csv
